In [1]:
import numpy as np
import pandas as pd

import sqlalchemy
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import Session
from sqlalchemy import create_engine, func
from sqlalchemy.sql import label
from sqlalchemy import MetaData

from flask import Flask, jsonify, render_template
from flask_sqlalchemy import SQLAlchemy

# pd.options.display.float_format = '${:10,.2f}'.format

# Mortgage concept 
~~~python
n = 30*12      # term in months
apr = 0.0426   # annual percentage rate
r = apr/12     # monthly percentage rate
baseloan = 203900.*0.8   # principal amount of loan
downpayment = 203900.*0.2
PV = baseloan - downpayment  # present value of the loan

per =              np.arange(1,n+1)  # months
payment =          np.pmt(r, n, PV)  # monthly payment
interestpayment =  np.ipmt(r, per, n, PV)  # monthly interest
principalpayment = np.ppmt(r, per, n, PV)  # monthly principal
balance =          PV + per*payment - sum(interestpayment) # monthly balance
dates =            pd.date_range('20181101', periods=n, freq='M')

df = pd.DataFrame({'date':dates,
                'month':per,
                'balance':balance,
                'payment':np.ones(len(per))*payment,
                'interest':interestpayment,
                'principal':principalpayment})

### reorder the columns to match the spreadsheet

pd.options.display.float_format = '${:10,.2f}'.format
df = df[['date','month','balance','payment','interest','principal']]
~~~

In [2]:
# ----------------------------------------------
# ---------   If import csv directly   ---------
# ----------------------------------------------
# df_mortgage = pd.read_csv("./data_csv/30yearFixedMortgage.csv")
# df_homeprice = pd.read_csv("./data_csv/State_MedianListingPrice_AllHomes.csv")
# df_oesm = pd.read_csv("./data_csv/state_M2017_dl.csv")


# ----------------------------------------------
# ---------   If import from sqlite   ----------
# ----------------------------------------------
app = Flask(__name__)
app.config["SQLALCHEMY_DATABASE_URI"] = "sqlite:///db/project_2.sqlite"
db = SQLAlchemy(app)

# reflect an existing database into a new model
Base = automap_base()

# reflect the tables
Base.prepare(db.engine, reflect=True)

# Save references to each table
home_price = Base.classes.home_price
mortgage_30 = Base.classes.mortgage_30
oesm = Base.classes.oesm

# Use Pandas to perform the sql query
stmt = db.session.query(mortgage_30).statement
df_mortgage = pd.read_sql_query(stmt, db.session.bind)
stmt = db.session.query(home_price).statement
df_homeprice = pd.read_sql_query(stmt, db.session.bind)
stmt = db.session.query(oesm).statement
df_oesm = pd.read_sql_query(stmt, db.session.bind)

/Users/cchuang/anaconda3/lib/python3.6/site-packages/flask_sqlalchemy/__init__.py:794: FSADeprecationWarning: SQLALCHEMY_TRACK_MODIFICATIONS adds significant overhead and will be disabled by default in the future.  Set it to True or False to suppress this warning.
  'SQLALCHEMY_TRACK_MODIFICATIONS adds significant overhead and '


# Mortgage related & Occupational Employment Data

## 1. mortgage_30
---
Average 30-year Mortgage Rates by State <br>
*REF: https://www.valuepenguin.com/mortgages/average-mortgage-rates#nogo* <br>
Rates assume a loan amount of $200,000 and a loan-to-value ratio of 80%.


## 2. home_price
---
State Median Listing Price <br>
*REF: https://www.zillow.com/research/data/* <br>
There are current and historical for-sale listing data, generally from 2010-01 to 2018-08. <br>

## 3. occupational employment (Bureau of Labor Statistics, Department of Labor)
---
May 2017 National Occupational Employment and Wage Estimates in each state <br>
*REF: https://www.bls.gov/oes/current/oes_nat.htm#11-0000* <br>
These estimates are calculated with data collected from employers in all industry sectors in every state and the District of Columbia. Additional information, including the hourly and annual 10th, 25th, 75th, and 90th percentile wages.<br><br>
**Major Occupational Groups (OCC_CODE)** <br>
- 00-0000  All Occupations
- 11-0000  Management Occupations
- 13-0000  Business and Financial Operations Occupations
- 15-0000  Computer and Mathematical Occupations
- 17-0000  Architecture and Engineering Occupations
- 19-0000  Life, Physical, and Social Science Occupations
- 21-0000  Community and Social Service Occupations
- 23-0000  Legal Occupations
- 25-0000  Education, Training, and Library Occupations
- 27-0000  Arts, Design, Entertainment, Sports, and Media Occupations
- 29-0000  Healthcare Practitioners and Technical Occupations
- 31-0000  Healthcare Support Occupations
- 33-0000  Protective Service Occupations
- 35-0000  Food Preparation and Serving Related Occupations
- 37-0000  Building and Grounds Cleaning and Maintenance Occupations
- 39-0000  Personal Care and Service Occupations
- 41-0000  Sales and Related Occupations
- 43-0000  Office and Administrative Support Occupations
- 45-0000  Farming, Fishing, and Forestry Occupations
- 47-0000  Construction and Extraction Occupations
- 49-0000  Installation, Maintenance, and Repair Occupations
- 51-0000  Production Occupations
- 53-0000  Transportation and Material Moving Occupations
<br><br>

**Field Description** <br>
- area: State FIPS code
- st: State abbreviation
- state: State name
- occ_code: The 6-digit Standard Occupational Classification (SOC) code or OES-specific code for the occupation 
- occ_title: Standard Occupational Classification title or OES-specific title for the occupation
- occ_group: Shows the SOC occupation level: "total"=total of all occupations; "major"=SOC major group; "minor"=SOC minor group; "broad"=SOC broad occupation; "detailed"=SOC detailed occupation
- tot_emp: Estimated total employment rounded to the nearest 10 (excludes self-employed)
- emp_prse: Percent relative standard error (RSE) for the employment. Relative standard error is a measure of the reliability of a statistic; the smaller the relative standard error, the more precise the estimate.
- jobs_1000: The number of jobs (employment) in the given occupation per 1,000 jobs in the given area (only on the statewide, metropolitan, and nonmetropolitan area files)
- loc_q: The location quotient represents the ratio of an occupation’s share of employment in a given area to that occupation’s share of employment in the U.S. as a whole. For example, an occupation that makes up 10 percent of employment in a specific metropolitan area compared with 2 percent of U.S. employment would have a location quotient of 5 for the area in question.
- h_mean: Mean hourly wage
- a_mean: Mean annual wage
- mean_prse: Percent relative standard error (RSE) for the mean wage. Relative standard error is a measure of the reliability of a statistic; the smaller the relative standard error, the more precise the estimate.
- h_pct10: Hourly 10th percentile wage
- h_pct25: Hourly 25th percentile wage
- h_median: Hourly median wage (or 50th percentile wage)
- h_pct75: Hourly 75th percentile wage
- h_pct90: Hourly 90th percentile wage
- a_pct10: Annual 10th percentile wage
- a_pct25: Annual 25th percentile wage
- a_median: Annual median wage (or 50th percentile wage)
- a_pct75: Annual 75th percentile wage
- a_pct90: Annual 90th percentile wage
- annual: Contains "TRUE" if only the annual wages are released. The OES program releases only annual wages for some occupations that typically work fewer than 2,080 hours per year but are paid on an annual basis, such as teachers, pilots, and athletes.
- hourly: Contains "TRUE" if only the hourly wages are released. Some occupations, such as actors, dancers, and musicians and singers, are paid hourly and generally don't work a standard 2,080 hour work year.


# Show data details

In [3]:
df_oesm.columns

Index(['Id', 'AREA', 'ST', 'STATE', 'OCC_CODE', 'OCC_TITLE', 'OCC_GROUP',
       'TOT_EMP', 'EMP_PRSE', 'JOBS_1000', 'LOC_Q', 'H_MEAN', 'A_MEAN',
       'MEAN_PRSE', 'H_PCT10', 'H_PCT25', 'H_MEDIAN', 'H_PCT75', 'H_PCT90',
       'A_PCT10', 'A_PCT25', 'A_MEDIAN', 'A_PCT75', 'A_PCT90', 'ANNUAL',
       'HOURLY'],
      dtype='object')

In [4]:
len(df_oesm)

36992

In [5]:
df_homeprice = df_homeprice.sort_values(['RegionName'])
len(df_homeprice)

51

In [6]:
df_mortgage[1:].head()

,state,average_rate,range_min,range_max
1,Alabama,4.26%,3.75%,5.13%
2,Alaska,4.34%,4.00%,4.75%
3,Arizona,4.39%,3.63%,5.00%
4,Arkansas,4.32%,3.88%,5.00%
5,California,4.44%,3.88%,5.00%


# Data use for house price with time/map

In [7]:
import sys
map_json_houseprice = {}
for t in np.arange(2,105,1):
    timeindex = df_homeprice.columns[t]
    map_json_houseprice.setdefault(timeindex, {})
    statelist = []
    valuelist = []
    for no, i in enumerate(df_homeprice['RegionName']):
        statelist.append(i)
        valuelist.append(df_homeprice.iloc[no,t])
    map_json_houseprice[timeindex] = {'state': statelist, 'value': valuelist}

In [8]:
# == Usage ==
# map_json_houseprice['2010-02']

# Data use for computing monthly payment 

In [9]:
df_housemerge = df_mortgage[1:].merge(df_homeprice, left_on="state", right_on="RegionName", how="outer")
df_housemerge = df_housemerge.drop(columns = ["RegionName"])
df_housemerge.replace('', np.nan, inplace=True)

In [10]:
df_housemerge.head()

,state,average_rate,range_min,range_max,SizeRank,2010-01,2010-02,2010-03,2010-04,2010-05,...,2017-11,2017-12,2018-01,2018-02,2018-03,2018-04,2018-05,2018-06,2018-07,2018-08
0,Alabama,4.26%,3.75%,5.13%,23,173900,173900,170000,169900,169900,...,191900,189900,190000,196000,199000,201900,204852,205000,203130.5,203900
1,Alaska,4.34%,4.00%,4.75%,47,250000,250000,255000,256450,264900,...,273700,269900,269900,274900,275500,285000,288600,287000,289000,284900
2,Arizona,4.39%,3.63%,5.00%,16,181984,179900,175900,175000,171338,...,275000,275000,275000,279000,279900,284000,285000,285000,284900,284000
3,Arkansas,4.32%,3.88%,5.00%,32,149900,149900,149900,149900,149900,...,165000,164000,164900,165000,169000,173950,174530,174900,175000,172500
4,California,4.44%,3.88%,5.00%,1,330000,335000,336900,339900,340900,...,499999,499000,499000,5e+05,519950,529900,539000,544000,539500,535000


In [11]:
avghomeprice = {}
timelist = []
avgprice = []
for i in np.arange(5,108,1):
    timelist.append( df_housemerge.columns[i] )
    avgprice.append( np.nanmean(np.array(df_housemerge.iloc[:,i].get_values(),dtype=float)))
avghomeprice = { 'time': timelist, \
                 'avghomeprice': avgprice}

In [12]:
df_monthly_payment = df_housemerge[['state','average_rate']].copy()
for i in np.arange(5,108,1):
    timeindex = df_housemerge.columns[i]
    monpay = []
    for index, row in df_housemerge.iterrows():
        monpay.append(-1.* np.pmt(0.01 * float(row['average_rate'].rstrip('%')) / 12., 
                         360., float(row[timeindex]) * 0.8))
    df_monthly_payment[timeindex] = monpay

In [13]:
df_monthly_payment.head()

avghomeprice = {}
timelist = []
avgprice = []
for i in np.arange(5,109,1):
    timelist.append( df_housemerge.columns[i] )
    avgprice.append( np.nanmean(np.array(df_housemerge.iloc[:,i].get_values(),dtype=float)))
avghomeprice = { 'time': timelist, \
                 'avghomeprice': avgprice}

df_monthly_payment = df_housemerge[['state','average_rate']].copy()
for i in np.arange(5,108,1):
    timeindex = df_housemerge.columns[i]
    monpay = []
    for index, row in df_housemerge.iterrows():
        monpay.append(-1.* np.pmt(0.01 * float(row['average_rate'].rstrip('%')) / 12., 
                         360., float(row[timeindex]) * 0.8))
    df_monthly_payment[timeindex] = monpay

In [14]:
df_monthly_payment.head()

,state,average_rate,2010-01,2010-02,2010-03,2010-04,2010-05,2010-06,2010-07,2010-08,...,2017-10,2017-11,2017-12,2018-01,2018-02,2018-03,2018-04,2018-05,2018-06,2018-07
0,Alabama,4.26%,685.201470,685.201470,669.834674,669.440654,669.440654,669.440654,665.894470,649.739634,...,768.339773,756.125141,748.244733,748.638753,772.279977,784.100589,795.527181,807.158663,807.741813,800.375602
1,Alaska,4.34%,994.446040,994.446040,1014.334961,1020.102748,1053.715024,1054.112803,1033.826103,1033.826103,...,1093.890644,1088.719525,1073.603945,1073.603945,1093.492866,1095.879536,1133.668486,1147.988509,1141.624054,1149.579623
2,Arizona,4.39%,728.184062,719.845221,703.839769,700.238542,685.585550,676.230363,659.824775,648.220822,...,1100.370850,1100.374851,1100.374851,1100.374851,1116.380304,1119.981531,1136.387119,1140.388482,1140.388482,1139.988346
3,Arkansas,4.32%,594.858996,594.858996,594.858996,594.858996,594.858996,595.255834,594.858996,594.858996,...,654.781417,654.781417,650.813045,654.384580,654.781417,670.654906,690.298348,692.600004,694.068302,694.465139
4,California,4.44%,1328.253943,1348.379003,1356.026525,1368.101561,1372.126573,1388.629122,1366.491556,1368.101561,...,2043.901067,2012.501949,2008.480962,2008.480962,2012.505974,2092.804962,2132.853831,2169.481440,2189.606500,2171.493946


# Data use for computing monthly payment 
## For plot bar chart

In [15]:
df_monthlypay_range = df_mortgage[1:].copy()
df_monthlypay_range = df_monthlypay_range.reset_index()

In [16]:
df_monthlypay_range['2018-07_houseprice'] = df_housemerge[['2018-07'][:]]
df_monthlypay_range.head()

,index,state,average_rate,range_min,range_max,2018-07_houseprice
0,1,Alabama,4.26%,3.75%,5.13%,203130.5
1,2,Alaska,4.34%,4.00%,4.75%,289000
2,3,Arizona,4.39%,3.63%,5.00%,284900
3,4,Arkansas,4.32%,3.88%,5.00%,175000
4,5,California,4.44%,3.88%,5.00%,539500


In [17]:
avg_pay = []
for index, row in df_monthlypay_range.iterrows():
    avg_pay.append( -1.* np.pmt(0.01 * float(row['average_rate'].rstrip('%')) / 12., 
                         360., float(row['2018-07_houseprice']) * 0.8) )
df_monthlypay_range['avg_monthlypay'] = avg_pay

low_pay = []
for index, row in df_monthlypay_range.iterrows():
    low_pay.append( -1.* np.pmt(0.01 * float(row['range_min'].rstrip('%')) / 12., 
                         360., float(row['2018-07_houseprice']) * 0.8) )
df_monthlypay_range['low_monthlypay'] = low_pay

high_pay = []
for index, row in df_monthlypay_range.iterrows():
    high_pay.append( -1.* np.pmt(0.01 * float(row['range_max'].rstrip('%')) / 12., 
                         360., float(row['2018-07_houseprice']) * 0.8) )
df_monthlypay_range['high_monthlypay'] = high_pay

In [18]:
df_monthlypay_range.head()

,index,state,average_rate,range_min,range_max,2018-07_houseprice,avg_monthlypay,low_monthlypay,high_monthlypay
0,1,Alabama,4.26%,3.75%,5.13%,203130.5,800.375602,752.583213,885.315291
1,2,Alaska,4.34%,4.00%,4.75%,289000,1149.579623,1103.784163,1206.048642
2,3,Arizona,4.39%,3.63%,5.00%,284900,1139.988346,1040.073648,1223.523843
3,4,Arkansas,4.32%,3.88%,5.00%,175000,694.465139,658.732377,751.550272
4,5,California,4.44%,3.88%,5.00%,539500,2171.493946,2030.777814,2316.922125


In [19]:
df_monthlypay_lowgroup = df_monthlypay_range.copy()
df_monthlypay_highgroup = df_monthlypay_range.copy()

In [20]:
df_monthlypay_lowgroup = df_monthlypay_lowgroup.sort_values(by=['avg_monthlypay'], ascending=True)
df_monthlypay_lowgroup = df_monthlypay_lowgroup.reset_index()

In [21]:
df_monthlypay_lowgroup.head(10)

,level_0,index,state,average_rate,range_min,range_max,2018-07_houseprice,avg_monthlypay,low_monthlypay,high_monthlypay
0,48,49,West Virginia,4.48%,3.88%,7.39%,159000,642.993009,598.505417,879.839532
1,35,36,Ohio,4.34%,3.79%,6.50%,169900,675.825529,632.555734,859.106858
2,3,4,Arkansas,4.32%,3.88%,5.00%,175000,694.465139,658.732377,751.550272
3,15,16,Iowa,4.32%,3.75%,4.75%,179000,710.338628,663.181527,746.998986
4,25,26,Missouri,4.36%,3.88%,5.00%,180000,717.697319,677.553302,773.023137
5,36,37,Oklahoma,4.38%,3.99%,5.48%,182900,730.985768,697.710770,828.953731
6,24,25,Mississippi,4.34%,4.00%,4.75%,183900,731.514507,702.373383,767.447561
7,14,15,Indiana,4.48%,3.75%,7.00%,181070.5,732.245696,670.852574,963.733244
8,17,18,Kentucky,4.38%,3.75%,5.00%,184900,738.979052,685.040583,794.066545
9,22,23,Michigan,4.41%,3.75%,4.88%,185000,742.000638,685.411076,783.677398


In [22]:
df_monthlypay_highgroup = df_monthlypay_highgroup.sort_values(by=['avg_monthlypay'], ascending=False)
df_monthlypay_highgroup = df_monthlypay_highgroup.reset_index()

In [23]:
df_monthlypay_highgroup.head()

,level_0,index,state,average_rate,range_min,range_max,2018-07_houseprice,avg_monthlypay,low_monthlypay,high_monthlypay
0,11,12,Hawaii,4.33%,3.88%,4.88%,615000,2443.442225,2314.973783,2605.197836
1,8,9,District of Columbia,4.36%,3.88%,4.88%,585000,2332.516287,2202.048232,2478.115015
2,4,5,California,4.44%,3.88%,5.00%,539500,2171.493946,2030.777814,2316.922125
3,21,22,Massachusetts,4.38%,3.75%,4.88%,447000,1786.498843,1656.101355,1893.534037
4,5,6,Colorado,4.38%,3.88%,4.75%,419000,1674.592875,1577.193520,1748.561872


In [24]:
output_boxchart = {}
output_boxchart.setdefault('high10',[])
output_boxchart.setdefault('low10',[])
for i in range(0,10):
    hh = df_monthlypay_highgroup.high_monthlypay[i]
    mm = df_monthlypay_highgroup.avg_monthlypay[i]
    ll = df_monthlypay_highgroup.low_monthlypay[i]
    output_boxchart['high10'].append( {'num': i,
                                 'states': df_monthlypay_highgroup.state[i],
                                 'avgmonthlypay': mm,
                                 'highbound': hh-mm,
                                 'lowbound': mm-ll } )

    hh = df_monthlypay_lowgroup.high_monthlypay[i]
    mm = df_monthlypay_lowgroup.avg_monthlypay[i]
    ll = df_monthlypay_lowgroup.low_monthlypay[i]
    output_boxchart['low10'].append( {'num': i,
                                 'states': df_monthlypay_lowgroup.state[i],
                                 'avgmonthlypay': mm,
                                 'highbound': hh-mm,
                                 'lowbound': mm-ll } )

# Data use for Occupational Employment Statistics (state - occupation)

In [25]:
df_housemerge = df_mortgage[1:].merge(df_homeprice, left_on="state", right_on="RegionName", how="outer")
df_housemerge = df_housemerge.drop(columns = ["RegionName"])
df_housemerge.replace('', np.nan, inplace=True)

In [26]:
df_oes = df_oesm[['ST','STATE','OCC_CODE','OCC_TITLE','TOT_EMP','JOBS_1000','H_MEAN','H_PCT10','H_PCT25','H_MEDIAN','H_PCT75','H_PCT90']].copy()
df_oes = df_oes[df_oes['OCC_CODE'].str.contains("-0000")]
df_oes.head()

,ST,STATE,OCC_CODE,OCC_TITLE,TOT_EMP,JOBS_1000,H_MEAN,H_PCT10,H_PCT25,H_MEDIAN,H_PCT75,H_PCT90
0,AL,Alabama,00-0000,All Occupations,"1,922,570",1000.000,20.76,8.54,10.45,15.77,25.01,37.83
1,AL,Alabama,11-0000,Management Occupations,"69,950",36.385,53.44,25.06,33.71,46.63,64.12,90.80
34,AL,Alabama,13-0000,Business and Financial Operations Occupations,"73,110",38.028,34.33,17.24,22.97,30.94,41.60,53.51
63,AL,Alabama,15-0000,Computer and Mathematical Occupations,"39,760",20.680,39.36,20.26,27.23,37.30,49.05,61.78
79,AL,Alabama,17-0000,Architecture and Engineering Occupations,"43,010",22.370,42.41,20.87,29.10,40.22,54.73,68.21


# NOTE

## The 28/36 Rule states that a household should spend a maximum of 28% of its gross monthly income on total housing expenses and no more than 36% on total debt service, including housing and other debt such as car loans.

# Here, we uses 30% of pre-tax income to compute the work hours.

In [27]:
"""
Remeber:
The 28/36 Rule states that a household should spend a maximum of 28% 
of its gross monthly income on total housing expenses and no more than 
36% on total debt service, including housing and other debt such as car loans.

Here, we uses 30% of pre-tax income to compute the work hours.
"""

July2018 = df_monthly_payment[['state','2018-07']]

df_occ_general_July2018 = df_oes.merge(July2018, left_on="STATE", right_on="state", how="outer")
df_occ_general_July2018 = df_occ_general_July2018.drop(columns = ['state'])


df_occ_general_July2018.rename(columns={'2018-07':'2018-07_monthlypay'}, inplace=True)

df_occ_general_July2018_clean = df_occ_general_July2018.apply(pd.to_numeric, errors='coerce')
df_occ_general_July2018_clean['ST'] = df_occ_general_July2018['ST']
df_occ_general_July2018_clean['STATE'] = df_occ_general_July2018['STATE']
df_occ_general_July2018_clean['OCC_CODE'] = df_occ_general_July2018['OCC_CODE']
df_occ_general_July2018_clean['OCC_TITLE'] = df_occ_general_July2018['OCC_TITLE']
df_occ_general_July2018_clean['TOT_EMP'] = df_occ_general_July2018['TOT_EMP']

df_occ_general_July2018_clean['mean_work_hour'] = round(pd.to_numeric(df_occ_general_July2018_clean['2018-07_monthlypay'])/(pd.to_numeric(df_occ_general_July2018_clean['H_MEAN'])*0.3))
df_occ_general_July2018_clean['PCT10_work_hour'] = round(pd.to_numeric(df_occ_general_July2018_clean['2018-07_monthlypay'])/(pd.to_numeric(df_occ_general_July2018_clean['H_PCT10'])*0.3))
df_occ_general_July2018_clean['PCT25_work_hour'] = round(pd.to_numeric(df_occ_general_July2018_clean['2018-07_monthlypay'])/(pd.to_numeric(df_occ_general_July2018_clean['H_PCT25'])*0.3))
df_occ_general_July2018_clean['PCT50_work_hour'] = round(pd.to_numeric(df_occ_general_July2018_clean['2018-07_monthlypay'])/(pd.to_numeric(df_occ_general_July2018_clean['H_MEDIAN'])*0.3))
df_occ_general_July2018_clean['PCT75_work_hour'] = round(pd.to_numeric(df_occ_general_July2018_clean['2018-07_monthlypay'])/(pd.to_numeric(df_occ_general_July2018_clean['H_PCT75'])*0.3))
df_occ_general_July2018_clean['PCT90_work_hour'] = round(pd.to_numeric(df_occ_general_July2018_clean['2018-07_monthlypay'])/(pd.to_numeric(df_occ_general_July2018_clean['H_PCT90'])*0.3))

df_occ_general_July2018_clean.head()

,ST,STATE,OCC_CODE,OCC_TITLE,TOT_EMP,JOBS_1000,H_MEAN,H_PCT10,H_PCT25,H_MEDIAN,H_PCT75,H_PCT90,2018-07_monthlypay,mean_work_hour,PCT10_work_hour,PCT25_work_hour,PCT50_work_hour,PCT75_work_hour,PCT90_work_hour
0,AL,Alabama,00-0000,All Occupations,"1,922,570",1000.000,20.76,8.54,10.45,15.77,25.01,37.83,800.375602,129.0,312.0,255.0,169.0,107.0,71.0
1,AL,Alabama,11-0000,Management Occupations,"69,950",36.385,53.44,25.06,33.71,46.63,64.12,90.80,800.375602,50.0,106.0,79.0,57.0,42.0,29.0
2,AL,Alabama,13-0000,Business and Financial Operations Occupations,"73,110",38.028,34.33,17.24,22.97,30.94,41.60,53.51,800.375602,78.0,155.0,116.0,86.0,64.0,50.0
3,AL,Alabama,15-0000,Computer and Mathematical Occupations,"39,760",20.680,39.36,20.26,27.23,37.30,49.05,61.78,800.375602,68.0,132.0,98.0,72.0,54.0,43.0
4,AL,Alabama,17-0000,Architecture and Engineering Occupations,"43,010",22.370,42.41,20.87,29.10,40.22,54.73,68.21,800.375602,63.0,128.0,92.0,66.0,49.0,39.0


In [28]:
output = []
for i in range(0, len(df_occ_general_July2018_clean)):
    if df_occ_general_July2018_clean.OCC_TITLE[i][:-12] == 'All':
        continue
    else:
        output.append( {'state': df_occ_general_July2018_clean.STATE[i], \
                    'occupation': df_occ_general_July2018_clean.OCC_TITLE[i][:-12], \
                    'total_employment': int(df_occ_general_July2018_clean.TOT_EMP[i].replace(',','')), \
                    'mean_work_hour': df_occ_general_July2018_clean.mean_work_hour[i], \
                    'PCT10_work_hour': df_occ_general_July2018_clean.PCT10_work_hour[i], \
                    'PCT25_work_hour': df_occ_general_July2018_clean.PCT25_work_hour[i], \
                    'PCT50_work_hour': df_occ_general_July2018_clean.PCT50_work_hour[i], \
                    'PCT75_work_hour': df_occ_general_July2018_clean.PCT75_work_hour[i], \
                    'PCT90_work_hour': df_occ_general_July2018_clean.PCT90_work_hour[i]} )

# Data use for computing work hour 

The 28/36 Rule states that a household should spend a maximum of 28% 
of its gross monthly income on total housing expenses and no more than 
36% on total debt service, including housing and other debt such as car loans.

Here, we uses 30% of pre-tax income to compute the work hours.

In [29]:
df_occ = df_oesm[['ST','STATE','OCC_CODE','OCC_TITLE','H_MEAN','H_PCT10','H_PCT25','H_MEDIAN','H_PCT75','H_PCT90']].copy()

In [30]:
df_occ.head()

,ST,STATE,OCC_CODE,OCC_TITLE,H_MEAN,H_PCT10,H_PCT25,H_MEDIAN,H_PCT75,H_PCT90
0,AL,Alabama,00-0000,All Occupations,20.76,8.54,10.45,15.77,25.01,37.83
1,AL,Alabama,11-0000,Management Occupations,53.44,25.06,33.71,46.63,64.12,90.80
2,AL,Alabama,11-1011,Chief Executives,99.56,40.63,60.24,94.45,#,#
3,AL,Alabama,11-1021,General and Operations Managers,58.04,25.98,34.91,49.19,71.09,#
4,AL,Alabama,11-1031,Legislators,*,*,*,*,*,*


In [31]:
df_occ_general = df_occ[df_occ['OCC_CODE'].str.contains("-0000")]
df_occ_general.head()

,ST,STATE,OCC_CODE,OCC_TITLE,H_MEAN,H_PCT10,H_PCT25,H_MEDIAN,H_PCT75,H_PCT90
0,AL,Alabama,00-0000,All Occupations,20.76,8.54,10.45,15.77,25.01,37.83
1,AL,Alabama,11-0000,Management Occupations,53.44,25.06,33.71,46.63,64.12,90.80
34,AL,Alabama,13-0000,Business and Financial Operations Occupations,34.33,17.24,22.97,30.94,41.60,53.51
63,AL,Alabama,15-0000,Computer and Mathematical Occupations,39.36,20.26,27.23,37.30,49.05,61.78
79,AL,Alabama,17-0000,Architecture and Engineering Occupations,42.41,20.87,29.10,40.22,54.73,68.21


In [32]:
"""
Remeber:
The 28/36 Rule states that a household should spend a maximum of 28% 
of its gross monthly income on total housing expenses and no more than 
36% on total debt service, including housing and other debt such as car loans.

Here, we uses 30% of pre-tax income to compute the work hours.
"""

July2018 = df_monthly_payment[['state','2018-07']]

df_occ_general_July2018 = df_occ_general.merge(July2018, left_on="STATE", right_on="state", how="outer")
df_occ_general_July2018 = df_occ_general_July2018.drop(columns = ['state'])


df_occ_general_July2018.rename(columns={'2018-07':'2018-07_monthlypay'}, inplace=True)

df_occ_general_July2018_clean = df_occ_general_July2018.apply(pd.to_numeric, errors='coerce')
df_occ_general_July2018_clean['ST'] = df_occ_general_July2018['ST']
df_occ_general_July2018_clean['STATE'] = df_occ_general_July2018['STATE']
df_occ_general_July2018_clean['OCC_CODE'] = df_occ_general_July2018['OCC_CODE']
df_occ_general_July2018_clean['OCC_TITLE'] = df_occ_general_July2018['OCC_TITLE']


df_occ_general_July2018_clean['mean_work_hour'] = round(pd.to_numeric(df_occ_general_July2018_clean['2018-07_monthlypay'])/(pd.to_numeric(df_occ_general_July2018_clean['H_MEAN'])*0.3))
df_occ_general_July2018_clean['PCT10_work_hour'] = round(pd.to_numeric(df_occ_general_July2018_clean['2018-07_monthlypay'])/(pd.to_numeric(df_occ_general_July2018_clean['H_PCT10'])*0.3))
df_occ_general_July2018_clean['PCT25_work_hour'] = round(pd.to_numeric(df_occ_general_July2018_clean['2018-07_monthlypay'])/(pd.to_numeric(df_occ_general_July2018_clean['H_PCT25'])*0.3))
df_occ_general_July2018_clean['PCT50_work_hour'] = round(pd.to_numeric(df_occ_general_July2018_clean['2018-07_monthlypay'])/(pd.to_numeric(df_occ_general_July2018_clean['H_MEDIAN'])*0.3))
df_occ_general_July2018_clean['PCT75_work_hour'] = round(pd.to_numeric(df_occ_general_July2018_clean['2018-07_monthlypay'])/(pd.to_numeric(df_occ_general_July2018_clean['H_PCT75'])*0.3))
df_occ_general_July2018_clean['PCT90_work_hour'] = round(pd.to_numeric(df_occ_general_July2018_clean['2018-07_monthlypay'])/(pd.to_numeric(df_occ_general_July2018_clean['H_PCT90'])*0.3))

df_occ_general_July2018_clean.head()

,ST,STATE,OCC_CODE,OCC_TITLE,H_MEAN,H_PCT10,H_PCT25,H_MEDIAN,H_PCT75,H_PCT90,2018-07_monthlypay,mean_work_hour,PCT10_work_hour,PCT25_work_hour,PCT50_work_hour,PCT75_work_hour,PCT90_work_hour
0,AL,Alabama,00-0000,All Occupations,20.76,8.54,10.45,15.77,25.01,37.83,800.375602,129.0,312.0,255.0,169.0,107.0,71.0
1,AL,Alabama,11-0000,Management Occupations,53.44,25.06,33.71,46.63,64.12,90.80,800.375602,50.0,106.0,79.0,57.0,42.0,29.0
2,AL,Alabama,13-0000,Business and Financial Operations Occupations,34.33,17.24,22.97,30.94,41.60,53.51,800.375602,78.0,155.0,116.0,86.0,64.0,50.0
3,AL,Alabama,15-0000,Computer and Mathematical Occupations,39.36,20.26,27.23,37.30,49.05,61.78,800.375602,68.0,132.0,98.0,72.0,54.0,43.0
4,AL,Alabama,17-0000,Architecture and Engineering Occupations,42.41,20.87,29.10,40.22,54.73,68.21,800.375602,63.0,128.0,92.0,66.0,49.0,39.0
